# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f4be8d5cbe0>
├── 'a' --> tensor([[-1.8513,  0.6966, -1.5605],
│                   [ 1.4228,  0.5973,  2.2907]])
└── 'x' --> <FastTreeValue 0x7f4be8d5cca0>
    └── 'c' --> tensor([[ 0.6544,  0.4686, -0.6769, -0.2901],
                        [ 0.2760,  0.5073,  0.0890, -0.0140],
                        [ 0.3960,  1.2854,  1.4938, -1.6303]])

In [4]:
t.a

tensor([[-1.8513,  0.6966, -1.5605],
        [ 1.4228,  0.5973,  2.2907]])

In [5]:
%timeit t.a

61.4 ns ± 0.0649 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f4be8d5cbe0>
├── 'a' --> tensor([[ 0.5644,  0.9767,  0.5906],
│                   [-0.8031, -1.3156,  1.3412]])
└── 'x' --> <FastTreeValue 0x7f4be8d5cca0>
    └── 'c' --> tensor([[ 0.6544,  0.4686, -0.6769, -0.2901],
                        [ 0.2760,  0.5073,  0.0890, -0.0140],
                        [ 0.3960,  1.2854,  1.4938, -1.6303]])

In [7]:
%timeit t.a = new_value

64 ns ± 0.0701 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.8513,  0.6966, -1.5605],
               [ 1.4228,  0.5973,  2.2907]]),
    x: Batch(
           c: tensor([[ 0.6544,  0.4686, -0.6769, -0.2901],
                      [ 0.2760,  0.5073,  0.0890, -0.0140],
                      [ 0.3960,  1.2854,  1.4938, -1.6303]]),
       ),
)

In [10]:
b.a

tensor([[-1.8513,  0.6966, -1.5605],
        [ 1.4228,  0.5973,  2.2907]])

In [11]:
%timeit b.a

57.7 ns ± 0.0222 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.4893,  0.7961, -0.6113],
               [ 1.4131,  0.9565,  0.9522]]),
    x: Batch(
           c: tensor([[ 0.6544,  0.4686, -0.6769, -0.2901],
                      [ 0.2760,  0.5073,  0.0890, -0.0140],
                      [ 0.3960,  1.2854,  1.4938, -1.6303]]),
       ),
)

In [13]:
%timeit b.a = new_value

496 ns ± 6.43 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

885 ns ± 12.3 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 27.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

135 µs ± 563 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

139 µs ± 781 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f4b1af7ff40>
├── 'a' --> tensor([[[-1.8513,  0.6966, -1.5605],
│                    [ 1.4228,  0.5973,  2.2907]],
│           
│                   [[-1.8513,  0.6966, -1.5605],
│                    [ 1.4228,  0.5973,  2.2907]],
│           
│                   [[-1.8513,  0.6966, -1.5605],
│                    [ 1.4228,  0.5973,  2.2907]],
│           
│                   [[-1.8513,  0.6966, -1.5605],
│                    [ 1.4228,  0.5973,  2.2907]],
│           
│                   [[-1.8513,  0.6966, -1.5605],
│                    [ 1.4228,  0.5973,  2.2907]],
│           
│                   [[-1.8513,  0.6966, -1.5605],
│                    [ 1.4228,  0.5973,  2.2907]],
│           
│                   [[-1.8513,  0.6966, -1.5605],
│                    [ 1.4228,  0.5973,  2.2907]],
│           
│                   [[-1.8513,  0.6966, -1.5605],
│                    [ 1.4228,  0.5973,  2.2907]]])
└── 'x' --> <FastTreeValue 0x7f4beae52970>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.3 µs ± 86.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f4b1af559d0>
├── 'a' --> tensor([[-1.8513,  0.6966, -1.5605],
│                   [ 1.4228,  0.5973,  2.2907],
│                   [-1.8513,  0.6966, -1.5605],
│                   [ 1.4228,  0.5973,  2.2907],
│                   [-1.8513,  0.6966, -1.5605],
│                   [ 1.4228,  0.5973,  2.2907],
│                   [-1.8513,  0.6966, -1.5605],
│                   [ 1.4228,  0.5973,  2.2907],
│                   [-1.8513,  0.6966, -1.5605],
│                   [ 1.4228,  0.5973,  2.2907],
│                   [-1.8513,  0.6966, -1.5605],
│                   [ 1.4228,  0.5973,  2.2907],
│                   [-1.8513,  0.6966, -1.5605],
│                   [ 1.4228,  0.5973,  2.2907],
│                   [-1.8513,  0.6966, -1.5605],
│                   [ 1.4228,  0.5973,  2.2907]])
└── 'x' --> <FastTreeValue 0x7f4b1af55a30>
    └── 'c' --> tensor([[ 0.6544,  0.4686, -0.6769, -0.2901],
                        [ 0.2760,  0.5073,  0.0890, -0.0140],
                 

In [23]:
%timeit t_cat(trees)

30.6 µs ± 54.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

78 µs ± 234 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-1.8513,  0.6966, -1.5605],
                [ 1.4228,  0.5973,  2.2907]],
       
               [[-1.8513,  0.6966, -1.5605],
                [ 1.4228,  0.5973,  2.2907]],
       
               [[-1.8513,  0.6966, -1.5605],
                [ 1.4228,  0.5973,  2.2907]],
       
               [[-1.8513,  0.6966, -1.5605],
                [ 1.4228,  0.5973,  2.2907]],
       
               [[-1.8513,  0.6966, -1.5605],
                [ 1.4228,  0.5973,  2.2907]],
       
               [[-1.8513,  0.6966, -1.5605],
                [ 1.4228,  0.5973,  2.2907]],
       
               [[-1.8513,  0.6966, -1.5605],
                [ 1.4228,  0.5973,  2.2907]],
       
               [[-1.8513,  0.6966, -1.5605],
                [ 1.4228,  0.5973,  2.2907]]]),
    x: Batch(
           c: tensor([[[ 0.6544,  0.4686, -0.6769, -0.2901],
                       [ 0.2760,  0.5073,  0.0890, -0.0140],
                       [ 0.3960,  1.2854,  1.4938, -1.6303]],
         

In [26]:
%timeit Batch.stack(batches)

79.9 µs ± 220 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-1.8513,  0.6966, -1.5605],
               [ 1.4228,  0.5973,  2.2907],
               [-1.8513,  0.6966, -1.5605],
               [ 1.4228,  0.5973,  2.2907],
               [-1.8513,  0.6966, -1.5605],
               [ 1.4228,  0.5973,  2.2907],
               [-1.8513,  0.6966, -1.5605],
               [ 1.4228,  0.5973,  2.2907],
               [-1.8513,  0.6966, -1.5605],
               [ 1.4228,  0.5973,  2.2907],
               [-1.8513,  0.6966, -1.5605],
               [ 1.4228,  0.5973,  2.2907],
               [-1.8513,  0.6966, -1.5605],
               [ 1.4228,  0.5973,  2.2907],
               [-1.8513,  0.6966, -1.5605],
               [ 1.4228,  0.5973,  2.2907]]),
    x: Batch(
           c: tensor([[ 0.6544,  0.4686, -0.6769, -0.2901],
                      [ 0.2760,  0.5073,  0.0890, -0.0140],
                      [ 0.3960,  1.2854,  1.4938, -1.6303],
                      [ 0.6544,  0.4686, -0.6769, -0.2901],
                      [ 0.2760,  

In [28]:
%timeit Batch.cat(batches)

150 µs ± 499 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

327 µs ± 4.79 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
